In [39]:
%matplotlib inline

In [40]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import matplotlib.style as style 

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import numpy as np

In [41]:
aggReports=pd.read_csv("../data/processed/clean_data.csv",index_col=0)
aggReports.head()

C:\Users\Rajasvi\Anaconda3\envs\ece143\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning:

Columns (1,6) have mixed types.Specify dtype option on import or set low_memory=False.



,report_id,caers_created_date,date_of_event,product_type,product,product_code,category,patient_age,age_units,sex,medra_preferred_terms,outcomes
0,172934,1/1/2014,12/5/2013,SUSPECT,VALUED NATIONAL PINE NUTS,23,Nuts/Edible Seed,NaN,NaN,NaN,"DYSGEUSIA, HYPERSENSITIVITY",Other Outcome
1,172937,1/1/2014,NaN,SUSPECT,"DAVID'S SUNFLOWER SEEDS, RANCH FLAVORED",23,Nuts/Edible Seed,NaN,NaN,NaN,"PAIN, MUCOSAL ULCERATION, TENDERNESS, BURNING ...","Hospitalization, Patient Visited Healthcare Pr..."
2,172939,1/1/2014,NaN,SUSPECT,"KASHI WHOLE WHEAT BISCUITS, ISLAND VANILLA",5,Cereal Prep/Breakfast Food,NaN,NaN,NaN,"VOMITING, ABDOMINAL PAIN, FEELING OF BODY TEMP...",Other Outcome
3,172940,1/1/2014,NaN,SUSPECT,DANNON DANNON LITE & FIT GREEK YOGURT CHERRY,9,Milk/Butter/Dried Milk Prod,NaN,NaN,NaN,NAUSEA,Other Outcome
4,172941,1/1/2014,12/16/2013,SUSPECT,COPPER RIVER KIPPERED ALASKA SILVER SALMON,16,Fishery/Seafood Prod,NaN,NaN,NaN,LACERATION,Other Outcome


In [8]:
uniq_out=set()
for o in aggReports.outcomes.unique():
    for i in o.split(","):
        if i!="":
            uniq_out.add(i.strip())
print(uniq_out)

for i in uniq_out:
    aggReports[i]=aggReports.apply(lambda x: True if i in x.outcomes else False,axis=1)



{'Patient Visited Healthcare Provider', 'Other Outcome', 'Other Seriousness', 'Death', 'Medically Important', 'Life Threatening', 'Patient Visited ER', 'Disability', 'Required Intervention', 'Hospitalization', 'Congenital Anomaly'}


In [41]:
aggReports.product_type

0             SUSPECT
1             SUSPECT
2             SUSPECT
3             SUSPECT
4             SUSPECT
             ...     
131256    CONCOMITANT
131257        SUSPECT
131258        SUSPECT
131259        SUSPECT
131260        SUSPECT
Name: product_type, Length: 131261, dtype: object

In [5]:
aggReports.outcomes = aggReports.outcomes.apply(lambda x: [y.strip() for y in x.split(",") if y!=[]])
expl_aggReports = aggReports.explode("outcomes")
expl_aggReports=expl_aggReports[["report_id","product","category","outcomes"]]
expl_aggReports=expl_aggReports.reset_index(drop=True)


In [37]:
expl_aggReports.outcomes.unique()

NameError: name 'expl_aggReports' is not defined

In [ ]:
def brand(x):
    stop = stopwords.words('english')
    n_x = [_.upper() for _ in x.lower().split(" ") if _ not in stop]
    if len(n_x)==0:
        return ""
    return n_x[0]  

def clean_prod(x):
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    x = regex.sub('', x)
    stop = stopwords.words('english')
    n_x = [_.upper() for _ in x.lower().split(" ") if _ not in stop]
    
    return " ".join(n_x) if len(n_x)<2 else " ".join(n_x[:2])


In [220]:
# Cosmetics Brand clustering
cosm_df = expl_aggReports[(expl_aggReports['category']=="Cosmetics") & (expl_aggReports['product']!="EXEMPTION 4")].copy()



cosm_df['brand'] =cosm_df['product'].apply(brand,1)

g=cosm_df.groupby(['outcomes','brand'])['report_id'].count().sort_values(ascending=False)


In [221]:
g.iloc[g.index.get_level_values('outcomes')=="Death"]

outcomes  brand     
Death     JOHNSONS      5
          JOHNSON       3
          FOLLIGEN      2
          WEN           1
          TWO           1
          SKIN          1
          SHOWER        1
          SAGE          1
          YOUNG         1
          PHILOSOPHY    1
          EMU           1
          MYSTIC        1
          MEDLINE       1
          L'OREAL       1
          DERMOLOGY     1
          COPPOLLA      1
          BRAZILIAN     1
Name: report_id, dtype: int64

In [222]:
cosm_df['outcomes'].unique()

array(['Disability', 'Medically Important', '', 'Other Outcome',
       'Life Threatening', 'Patient Visited Healthcare Provider',
       'Required Intervention', 'Hospitalization', 'Patient Visited ER',
       'Other Seriousness', 'Death', 'Congenital Anomaly'], dtype=object)

In [ ]:
fig = make_subplots(rows=3, cols=3)
r=1
c=1

relv_outcomes = ["Death","Medically Important","Life Threatening","Hospitalization","Disability","Patient Visited ER"]

for i,outcome in enumerate(relv_outcomes):
    
    top_brands = list(g.iloc[g.index.get_level_values('outcomes')==outcome].reset_index()[:5]['brand'])
    brand_count = list(g.iloc[g.index.get_level_values('outcomes')==outcome].reset_index()[:10]['report_id'])
    fig.add_trace(go.Bar(name=outcome, x = top_brands,y = brand_count),row=r, col = c)

    if c==3:
        c=1
        r+=1
    else:
        c+=1
fig.layout.update(title="Brand vs Outcome", height = 1500, width = 1500)
fig.show()
plot(fig, filename ="brand_vs_outcome.html")


In [224]:
# Cosmetics Brand vs Outcomes

data = []
relv_outcomes = ["Death","Life Threatening","Hospitalization","Disability","Patient Visited ER"]

for i,outcome in enumerate(relv_outcomes):
    
    top_brands = list(g.iloc[g.index.get_level_values('outcomes')==outcome].reset_index()[:5]['brand'])
    brand_count = list(g.iloc[g.index.get_level_values('outcomes')==outcome].reset_index()[:5]['report_id'])

    data.append(go.Bar(name=outcome, x = top_brands,y = brand_count))

fig = go.Figure(data=data)

fig.layout.update(title="Cosmetics Brands vs Outcomes", height = 500, width = 1500,barmode = "stack")
fig.show()
plot(fig, filename ="../reports/figures/brands_vs_outcomes_cosmetics.html")




'../reports/figures/brands_vs_outcomes_cosmetics.html'

In [225]:
# Vit/Min/Prot/Unconv Diet(Human/Animal)  Brand clustering

vit_df = expl_aggReports[(expl_aggReports['category']=="Vit/Min/Prot/Unconv Diet(Human/Animal)") & (expl_aggReports['product']!="EXEMPTION 4")].copy()

vit_df.dropna(inplace=True)


vit_df['brand'] =vit_df['product'].apply(clean_prod,1)

vit_g=vit_df.groupby(['outcomes','brand'])['report_id'].count().sort_values(ascending=False)



In [227]:
# Vitamins product vs Outcomes

data = []

relv_outcomes = ["Death","Life Threatening","Hospitalization","Disability","Patient Visited ER"]

for i,outcome in enumerate(relv_outcomes):
    
    top_brands = list(vit_g.iloc[vit_g.index.get_level_values('outcomes')==outcome].reset_index()[:5]['brand'])
    brand_count = list(vit_g.iloc[vit_g.index.get_level_values('outcomes')==outcome].reset_index()[:5]['report_id'])

    data.append(go.Bar(name=outcome, x = top_brands,y = brand_count))

fig = go.Figure(data=data)

fig.layout.update(title="Vitamins Products vs Outcomes", height = 500, width = 1500,barmode = "stack")
fig.show()
plot(fig, filename ="../reports/figures/brands_vs_outcomes_vit.html")




'../reports/figures/brands_vs_outcomes_vit.html'

In [228]:
# Vegetables/Vegetable Products Brand clustering
veg_df = expl_aggReports[(expl_aggReports['category']=="Vegetables/Vegetable Products") & (expl_aggReports['product']!="EXEMPTION 4")].copy()

veg_df.dropna(inplace=True)

def brand(x):
    stop = stopwords.words('english')
    n_x = [_.upper() for _ in x.lower().split(" ") if _ not in stop]
    if len(n_x)==0:
        return ""
    return n_x[0]  

def clean_prod(x):
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    x = regex.sub('', x)
    stop = stopwords.words('english')
    n_x = [_.upper() for _ in x.lower().split(" ") if _ not in stop]
    
    return " ".join(n_x) if len(n_x)<2 else " ".join(n_x[:2])

veg_df['brand'] =veg_df['product'].apply(brand,1)

veg_g=veg_df.groupby(['outcomes','brand'])['report_id'].count().sort_values(ascending=False)



In [229]:
veg_df[veg_df['brand']=="DOLE"]

,report_id,product,category,outcomes,brand
422,173158,"DOLE SPINACH, LEAF AND STEM VEGETABLE",Vegetables/Vegetable Products,Other Outcome,DOLE
474,173226,DOLE BRAND HEART OF ROMAINE LETTUCE,Vegetables/Vegetable Products,Hospitalization,DOLE
475,173226,DOLE BRAND HEART OF ROMAINE LETTUCE,Vegetables/Vegetable Products,Patient Visited ER,DOLE
476,173226,DOLE BRAND HEART OF ROMAINE LETTUCE,Vegetables/Vegetable Products,,DOLE
562,173293,DOLE SPRING MIX,Vegetables/Vegetable Products,Other Outcome,DOLE
...,...,...,...,...,...
243034,165063,DOLE SPRING MIX,Vegetables/Vegetable Products,Other Outcome,DOLE
243094,165068,DOLE SPRING MIX,Vegetables/Vegetable Products,Other Outcome,DOLE
246507,168555,DOLE ROMAINE LETTUCE SALAD MIX,Vegetables/Vegetable Products,Other Outcome,DOLE
249661,170698,DOLE DOLE CLASSIC ROMAINE LETTUCE,Vegetables/Vegetable Products,Patient Visited Healthcare Provider,DOLE


In [231]:
# Veg product vs Outcomes

data = []

relv_outcomes = ["Death","Life Threatening","Hospitalization","Disability","Patient Visited ER"]

for i,outcome in enumerate(relv_outcomes):
    
    top_brands = list(veg_g.iloc[veg_g.index.get_level_values('outcomes')==outcome].reset_index()[:5]['brand'])
    brand_count = list(veg_g.iloc[veg_g.index.get_level_values('outcomes')==outcome].reset_index()[:5]['report_id'])

    data.append(go.Bar(name=outcome, x = top_brands,y = brand_count))

fig = go.Figure(data=data)

fig.layout.update(title="Vegetable(s) Products vs Outcomes", height = 500, width = 1500,barmode = "stack")
fig.show()
plot(fig, filename ="../reports/figures/brands_vs_outcomes_veg.html")




'../reports/figures/brands_vs_outcomes_veg.html'

In [38]:
px.histogram()

NameError: name 'px' is not defined

In [42]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

unique_brand=list(set(cosm_df['product'].values))[:1000]

score_sort = [(x,) + i
             for x in unique_brand 
             for i in process.extract(x, unique_brand, scorer=fuzz.token_sort_ratio)]
similarity_sort = pd.DataFrame(score_sort, columns=['brand_sort','match_sort','score_sort'])

similarity_sort['sorted_brand_sort'] = np.minimum(similarity_sort['brand_sort'], similarity_sort['match_sort'])

high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 80) &
                (similarity_sort['brand_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_brand_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_brand_sort',axis=1).copy()

high_score_sort.groupby(['brand_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)[:20]

,,match_sort
brand_sort,score_sort,
MONAT REVIVE SHAMPOO,100,Monat Revive Shampoo
JACLYN HILL COSMETICS SO RICH LIPSTICK - ISLA,100,JACLYN HILL COSMETICS SO RICH LIPSTICK ISLA
DEVACURL NO POO ORIGINAL,98,DEVACURL NO POO ORIGINIAL
PANTENE SHAMPOO,97,PANTENE SHAMPOOS
PREMIER PIGMENTS TRUE COLOR CHAMPAGNE TATTOO,96,PREMIER PIGMENTS TRUE COLORS LT. CHAMPAGNE TATTOO
WEN CLEANSING CONDITIONER POMEGRANATE,95,WEN POMAGRANETE CLEANSING CONDITIONER
DEVACURL LOO POO DELIGHT SHAMPOO,95,DEVACURL LOWPOO DELIGHT SHAMPOO
DEVACURL DECADACE CONDITIONER,95,DEVACURL DECADENCE CONDITIONER
DEVACURL NO POO ORIGINAL,94,DEVACURL ORIGINAL LOW POO


In [31]:
# Exemption 4 split description-wise
aggReports[aggReports['product']=="EXEMPTION 4"].groupby("description")['report_id'].count().sort_values(ascending=False)

description
Cosmetics                                       17094
Vit/Min/Prot/Unconv Diet(Human/Animal)           1347
Dietary Conventional Foods/Meal Replacements      221
Baby Food Products                                 72
Fishery/Seafood Prod                               46
Soft Drink/Water                                   42
Bakery Prod/Dough/Mix/Icing                        38
Fruit/Fruit Prod                                   31
Nuts/Edible Seed                                   26
Vegetables/Vegetable Products                      25
Coffee/Tea                                         17
Cereal Prep/Breakfast Food                         16
Milk/Butter/Dried Milk Prod                        16
Snack Food Item                                    15
Whole Grain/Milled Grain Prod/Starch               15
Mult Food Dinner/Grav/Sauce/Special                12
Cheese/Cheese Prod                                  9
Choc/Cocoa Prod                                     8
Spices, Flavors 

In [18]:
# Cosmetics Detailed EDA
# Can Cosmetics actually kill you?

expl_aggReports[(expl_aggReports['description']=="Cosmetics")& (expl_aggReports['product']!="EXEMPTION 4")  ].groupby(["outcomes","product"])["report_id"].count().sort_values(ascending=False)[:20]

# All: D: 4931 H: 1083
# E4: D: 4907   H : 773
# WEN: D:0 H: 0


outcomes                             product                                    
                                     WEN CLEANSING CONDITIONER                      258
Medically Important                  WEN CLEANSING CONDITIONER                      211
Other Outcome                        WEN CLEANSING CONDITIONER                      152
                                     WEN                                            108
Medically Important                  WEN                                             94
Patient Visited Healthcare Provider  WEN CLEANSING CONDITIONER                       78
Other Outcome                        MONAT RENEW SHAMPOO                             72
                                     WEN                                             52
                                     MONAT RENEW SHAMPOO                             50
                                     WEN SWEET ALMOND MINT CLEANSING CONDITIONER     50
Patient Visited Healthcare Provider  WE

In [26]:
# Vitamin

g=expl_aggReports[(expl_aggReports['description']=="Vit/Min/Prot/Unconv Diet(Human/Animal)") & (expl_aggReports['product']!="EXEMPTION 4")].groupby(["outcomes","product"])["report_id"].count().sort_values(ascending=False)

g.iloc[g.index.get_level_values('outcomes')=="Death"]

# All: D: 1165 LT: 4905 H: 18365
# Beta: D: 2  H: 336 (Max)
# KRATOM: D: 124 (Max) H:  43

outcomes  product                                                  
Death     KRATOM                                                       124
          5-HOUR ENERGY                                                 12
          BENEFIBER WITH WHEAT DEXTRIN (NCH) (WHEAT DEXTRIN) POWDER     11
          5 HOUR ENERGY                                                 11
          CENTRUM                                                        9
                                                                      ... 
          DUAL ACTION CLEANSE TOTAL BODY PURIFIER                        1
          DUAL-ACTION CLEANSE COLON CLEAR FORMULA                        1
          EAS BETAGEN POWDER (EAS CREATINE POWDER)                       1
          EAS CREATINE                                                   1
          EDENS ETHNOS KRATOM                                            1
Name: report_id, Length: 773, dtype: int64

In [27]:
# description: broad category; group on product vs outcome

top_10=list(aggReports.groupby('description')['report_id'].count().sort_values(ascending=False).index[:10])

g=expl_aggReports.groupby(['description','outcomes'])['report_id'].count().sort_values(ascending=False)
g=expl_aggReports[expl_aggReports['description'].isin(top_10)]
sns.set_theme()
g.iloc[g.index.get_level_values('outcomes')=="Death"]

plt.figure(figsize=(10,10))
ax=sns.displot(expl_aggReports,x="outcomes",hue="description")
ax.set_xticklabels(rotation=90)


KeyError: 'Requested level (outcomes) does not match index name (None)'

In [35]:
# Veggies

g=expl_aggReports[(expl_aggReports['description']=="Vegetables/Vegetable Products") & (expl_aggReports['product']!="EXEMPTION 4")].groupby(["outcomes"])["report_id"].count().sort_values(ascending=False)
g
# g.iloc[g.index.get_level_values('outcomes')=="Hospitalization"]



# All: D: 1165 LT: 4905 H: 18365
# Beta: D: 2  H: 336 (Max)
# KRATOM: D: 124 (Max) H:  43

outcomes
Other Outcome                          2417
                                        607
Patient Visited Healthcare Provider     438
Patient Visited ER                      351
Medically Important                     213
Other Seriousness                       171
Hospitalization                         141
Disability                               87
Life Threatening                         79
Required Intervention                    47
Death                                    12
Congenital Anomaly                        1
Name: report_id, dtype: int64

In [28]:
#Check all pairs of category
s_product = aggReports.groupby(["description","product"],as_index=False)['report_id'].count().sort_values(['description','report_id'],ascending=False)

s_product[(s_product['description']=="Vegetables/Vegetable Products")].head(10)


,description,product,report_id
28377,Vegetables/Vegetable Products,QUORN,60
28502,Vegetables/Vegetable Products,QUORN NUGGETS,52
28418,Vegetables/Vegetable Products,QUORN CHICKEN PATTY,49
27132,Vegetables/Vegetable Products,CABBAGE,34
28414,Vegetables/Vegetable Products,QUORN CHICKEN CUTLETS,31
28603,Vegetables/Vegetable Products,ROMAINE LETTUCE,31
27172,Vegetables/Vegetable Products,CARROT,29
28402,Vegetables/Vegetable Products,QUORN CHICK'N NUGGETS,29
28484,Vegetables/Vegetable Products,QUORN MEATBALLS,28
27502,Vegetables/Vegetable Products,EXEMPTION 4,25


In [7]:
# product: specific sub-category 
aggReports.groupby('product')['report_id'].count().sort_values(ascending=False).head(10)

product
EXEMPTION 4                                                         19712
SUPER BETA PROSTATE                                                  1106
VITAMIN D                                                             838
MULTIVITAMIN                                                          723
FISH OIL                                                              701
CALCIUM                                                               659
CENTRUM SILVER WOMEN'S 50+ (MULTIMINERALS, MULTIVITAMINS) TABLET      618
VITAMIN C                                                             545
WEN CLEANSING CONDITIONER                                             413
MAGNESIUM                                                             355
Name: report_id, dtype: int64

In [25]:
# description: broad category
aggReports.groupby('description')['report_id'].count().sort_values(ascending=False).index[:10]

Index(['Vit/Min/Prot/Unconv Diet(Human/Animal)', 'Cosmetics',
       'Vegetables/Vegetable Products', 'Nuts/Edible Seed', 'Soft Drink/Water',
       'Bakery Prod/Dough/Mix/Icing', 'Fruit/Fruit Prod',
       'Fishery/Seafood Prod', 'Dietary Conventional Foods/Meal Replacements',
       'Milk/Butter/Dried Milk Prod'],
      dtype='object', name='description')

In [85]:
aggReports[aggReports['description']=="Nuts/Edible Seed"][uniq_out].sum().sort_values(ascending=False)

Other Outcome                          2503
Patient Visited Healthcare Provider     587
Patient Visited ER                      324
Hospitalization                         191
Medically Important                     153
Life Threatening                         68
Other Seriousness                        59
Required Intervention                    40
Disability                               35
Death                                    20
Congenital Anomaly                        3
dtype: int64

In [105]:
aggReports[aggReports['description']=="Cosmetics"][uniq_out].sum().sort_values(ascending=False)*100/27758

Medically Important                    54.921104
Patient Visited Healthcare Provider    23.506737
Other Outcome                          19.572736
Death                                  17.764248
Hospitalization                         3.901578
Patient Visited ER                      2.986526
Disability                              1.639167
Life Threatening                        0.929462
Other Seriousness                       0.691692
Required Intervention                   0.673680
Congenital Anomaly                      0.032423
dtype: float64

In [87]:
aggReports[aggReports['description']=="Cosmetics"][uniq_out].sum().sort_values(ascending=False)*100/27758

Medically Important                    54.921104
Patient Visited Healthcare Provider    23.506737
Other Outcome                          19.572736
Death                                  17.764248
Hospitalization                         3.901578
Patient Visited ER                      2.986526
Disability                              1.639167
Life Threatening                        0.929462
Other Seriousness                       0.691692
Required Intervention                   0.673680
Congenital Anomaly                      0.032423
dtype: float64

In [80]:
aggReports.groupby(["description"])['report_id'].count().sort_values(ascending=False)[:10]

description
Vit/Min/Prot/Unconv Diet(Human/Animal)          67596
Cosmetics                                       27758
Vegetables/Vegetable Products                    3581
Nuts/Edible Seed                                 3551
Soft Drink/Water                                 2908
Bakery Prod/Dough/Mix/Icing                      2905
Fruit/Fruit Prod                                 2622
Fishery/Seafood Prod                             2604
Dietary Conventional Foods/Meal Replacements     1767
Milk/Butter/Dried Milk Prod                      1680
Name: report_id, dtype: int64

In [76]:
aggReports[aggReports['product']=="EXEMPTION 4"].groupby(["product","description"])['report_id'].count().sort_values(ascending=False)[:10]

product      description                                 
EXEMPTION 4  Cosmetics                                       17094
             Vit/Min/Prot/Unconv Diet(Human/Animal)           1347
             Dietary Conventional Foods/Meal Replacements      221
             Baby Food Products                                 72
             Fishery/Seafood Prod                               46
             Soft Drink/Water                                   42
             Bakery Prod/Dough/Mix/Icing                        38
             Fruit/Fruit Prod                                   31
             Nuts/Edible Seed                                   26
             Vegetables/Vegetable Products                      25
Name: report_id, dtype: int64

In [59]:
aggReports[(aggReports['product']=="SUPER BETA PROSTATE")].groupby("outcomes")['report_id'].count().sort_values(ascending=False)

outcomes
Medically Important,                                                                                426
Hospitalization, Medically Important,                                                               129
Patient Visited Healthcare Provider, Medically Important,                                           112
Hospitalization,                                                                                     91
Medically Important, Patient Visited ER,                                                             80
Medically Important, Other Seriousness,                                                              46
Other Seriousness,                                                                                   43
Hospitalization, Medically Important, Patient Visited ER,                                            42
Hospitalization, Patient Visited ER,                                                                 19
Hospitalization, Other Seriousness,                    

In [41]:

def plot_bar(x,y):
        fig = plt.figure(figsize = (10, 5))
        #  Bar plot
        plt.bar(x, y, color ='green',
                width = 0.5)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.show()

In [64]:
(aggReports.isnull().sum()/aggReports.shape[0])*100

report_id                 0.000000
caers_created_date        0.000000
date_of_event            38.439445
product_type              0.000000
product                   0.002286
product_code              0.495958
description               0.495958
patient_age              39.411554
age_units                39.314038
sex                      40.522318
medra_preferred_terms     0.817455
outcomes                  0.000000
dtype: float64

In [57]:
len(aggReports)

131261

In [58]:
aggReports_no_dupes = aggReports.copy()
aggReports_no_dupes.drop_duplicates(subset = 'Report ID',inplace=True)

len(aggReports_no_dupes)

91776

In [60]:
(aggReports_no_dupes.isnull().sum()/aggReports.shape[0])*100

Report ID                  0.000000
CAERS Created Date         0.000000
Date of Event             28.853201
Product Type               0.000000
Product                    0.001524
Product Code               0.474627
Description                0.474627
Patient Age               33.466909
Age Units                 33.391487
Sex                       34.260748
MedDRA Preferred Terms    41.396150
Outcomes                   0.000000
Medra Preferred Terms     29.271452
dtype: float64

In [68]:
aggReports_no_dupes.nunique()

Report ID                 91776
CAERS Created Date         4927
Date of Event              6503
Product Type                  2
Product                   42497
Product Code                 48
Description                  50
Patient Age                 107
Age Units                     5
Sex                           4
MedDRA Preferred Terms    18984
Outcomes                    420
Medra Preferred Terms     23531
time_diff                  4785
time_group                    6
dtype: int64

In [63]:
aggReports_no_dupes["CAERS Created Date"] = pd.to_datetime(aggReports_no_dupes["CAERS Created Date"])
aggReports_no_dupes["Date of Event"] = pd.to_datetime(aggReports_no_dupes["Date of Event"])

aggReports_no_dupes["time_diff"] = (aggReports_no_dupes['CAERS Created Date']-aggReports_no_dupes['Date of Event']).dt.days

# Group the Time difference by different periods
time_list = (0,180,366,1098,1830,3660)
time_cat = ['less than 6 Months','less than 1 year','1 to 3 years','3 to 5 years','5 to 10 Years']
aggReports_no_dupes['time_group'] = pd.cut(aggReports_no_dupes['time_diff'],time_list,labels= time_cat)

aggReports_no_dupes['time_group'] = aggReports_no_dupes['time_group'].astype('str')
aggReports_no_dupes['time_group'] = aggReports_no_dupes['time_group'].replace('nan','Unknown')

In [72]:
aggReports_no_dupes.groupby('time_group')['Report ID'].count()

time_group
1 to 3 years           5304
3 to 5 years           2627
5 to 10 Years          2931
Unknown               40120
less than 1 year       3546
less than 6 Months    37248
Name: Report ID, dtype: int64

In [ ]:
def brand_product_plot( base_df, category,filename,title,first_word=True,trim_len=2):
    
    relv_outcomes = ["Death","Life Threatening","Hospitalization","Disability","Patient Visited ER"]
    data = []


    df = base_df[(base_df['category']==category) & (base_df['product']!="EXEMPTION 4")].copy()
    df.dropna(inplace=True)

    df['brand'] =df['product'].apply(brand_preprocess,first_word=first_word,trim_len=trim_len)
    df=df.groupby(['outcomes','brand'])['report_id'].count().sort_values(ascending=False)

    for i,outcome in enumerate(relv_outcomes):        
        top_brands = list(df.iloc[df.index.get_level_values('outcomes')==outcome].reset_index()[:5]['brand'])

        brand_count = list(df.iloc[df.index.get_level_values('outcomes')==outcome].reset_index()[:5]['report_id'])

        data.append(go.Bar(name=outcome, x = top_brands,y = brand_count))

    fig = go.Figure(data=data)
    fig.layout.update(title=f"{title} Products vs Outcomes", height = 500, width = 1500,barmode = "stack")

    fig.show()
    # plot(fig, filename =f"../reports/figures/brands_vs_outcomes_{filename}.html")

In [ ]:
aggReports